In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
training_data = pd.read_csv('/kaggle/input/word2vec-nlp-tutorial/labeledTrainData.tsv.zip',header=0,delimiter='\t',quoting=3)

In [ ]:
training_data.head()

In [ ]:
training_data['review'][0]

In [ ]:
#удаляем HTML разметку и тэги, такие как "<br>"
from bs4 import BeautifulSoup

In [ ]:
example1 = BeautifulSoup(training_data["review"][0]) 
example1.get_text()

In [ ]:
def review_to_words( raw_review ):
    # Функция для превращения обзора в строку слов
    # На вход необработанный обзор фильма 
    # На выходе обработанный
    #
    # 1. Удаляем HTML
    review_text = BeautifulSoup(raw_review).get_text() 
    #
    # 2. Удаляем не буквы       
    letters_only = re.sub("[^a-zA-Z]", " ", review_text) 
    #
    # 3. Преобразуем в lower-case, делим на отдельные слова
    words = letters_only.lower().split()                             
    #
    # 4. Для ускорения поиска преобразуем список в set
    stops = set(stopwords.words("english"))                  
    # 
    # 5. Удаляем стоповые слова
    meaningful_words = [w for w in words if not w in stops]   
    
    #6.Lemmatization
    for word in meaningful_words:
        word = wordnet_lemmatizer.lemmatize(word,'v')
    
    # 6. Соединим слова обратно в одну строку с разделителем " ", 
    # и вернем результат.
    return( " ".join( meaningful_words )) 

In [ ]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

Чистим и парсим тренировочный набор данных

In [ ]:
num_reviews = training_data['review'].size
clean_train_reviews = []
for i in range( 0, num_reviews ):                                                                  
    clean_train_reviews.append( review_to_words( training_data["review"][i] ))

In [ ]:
clean_train_reviews[0]

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
# Инициализируем объект "CountVectorizer", который является инструментом scikit-learn's bag of words.  
vectorizer = CountVectorizer(analyzer = "word",
                             tokenizer = None, 
                             preprocessor = None,
                             stop_words = None,  
                             max_features = 5000)

In [ ]:
train_data_features = vectorizer.fit_transform(clean_train_reviews)

In [ ]:
train_data_features = train_data_features.toarray()

In [ ]:
#25000 обзоров и 5000 уникальных слов(5000 - это число, которое мы назначили 
#в in max_features параметре функции CountVectorizer
#чтобы ограничить размер словаря уникальных слов до 5000)
train_data_features.shape

In [ ]:
#Используем классификатор RandomForest для классификации обзоров
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train_data_features, training_data["sentiment"], 
                                                    test_size=0.2)

In [ ]:
RF = RandomForestClassifier(n_estimators = 100)
RF.fit( X_train, y_train )

In [ ]:
predictions = RF.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix,classification_report

In [ ]:
print(confusion_matrix(y_test,predictions))

In [ ]:
print(classification_report(y_test,predictions))

In [ ]:
test_data = pd.read_csv('/kaggle/input/word2vec-nlp-tutorial/testData.tsv.zip',header=0,delimiter='\t',quoting=3)

In [ ]:
test_data.head()

In [ ]:
num_reviews = len(test_data["review"])
clean_test_reviews = [] 

Чистим и парсим тестовый набор данных

In [ ]:
for i in range(0,num_reviews):
    clean_review = review_to_words( test_data["review"][i] )
    clean_test_reviews.append( clean_review )

# Получаем набор слов для тестового набора, и превращаем его в numpy массив
test_data_features = vectorizer.transform(clean_test_reviews)
test_data_features = test_data_features.toarray()

In [ ]:
# Используем random forest для предсказания
result = RF.predict(test_data_features)

output = pd.DataFrame( data={"id":test_data["id"], "sentiment":result} )

output.to_csv( "Bag_of_Words_model.csv", index=False, quoting=3 )

Попытаемся улучшить результат при помощи нейронных сетей.

In [ ]:
import tensorflow as tf

In [ ]:
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense
from tensorflow.python import keras

In [ ]:
#Строим модель и компилируем ее
model = keras.Sequential([
    keras.layers.Dense(128,input_shape=(5000,),activation=tf.nn.relu),
    keras.layers.Dense(128,activation=tf.nn.relu),
    keras.layers.Dense(1, activation=tf.nn.sigmoid)
])

model.compile(optimizer='adam', 
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(X_train,y_train,epochs=10)

In [ ]:
predictions_ANN = model.predict(X_test)

In [ ]:
#Превращаем результат в 1 и 0
predictions_ANN = (predictions_ANN>0.9)

In [ ]:
predictions_ANN

In [ ]:
print(confusion_matrix(y_test,predictions_ANN))

In [ ]:
print(classification_report(y_test,predictions_ANN))

In [ ]:
result_ANN = model.predict(test_data_features)
result_ANN = result_ANN>0.9

In [ ]:
result_ANN

In [ ]:
result_ANN = [1 if res==True else 0 for res in result_ANN]

In [ ]:
output = pd.DataFrame( data={"id":test_data["id"], "sentiment":result_ANN} )

#2 вариант предсказания
output.to_csv( "Bag_of_Words_model_ANN.csv", index=False, quoting=3 )